In [1]:
import tabula
import textract
import re
import pymysql
from os import listdir
from os.path import isfile, join
import pandas as pd
from IPython.display import display
import db_conn

In [2]:
CONN = db_conn.get_connection()

In [3]:
def get_country(country):
    curs = CONN.cursor(pymysql.cursors.DictCursor)
    
    find_sql = '''select id from team where lower(country) like %s'''
    curs.execute(find_sql, ('%'+country+'%'))
    result = curs.fetchone()
    return result

def get_player_id(country_id, num):
#     print(str(country_id)+';;;;'+str(num))
    curs = CONN.cursor(pymysql.cursors.DictCursor)
    
    find_sql = '''select * from player where team_id=%s and num=%s'''
    curs.execute(find_sql, (country_id, num))
    result = curs.fetchone()
    return result    

def save_stat(player_stat):
    print(player_stat)
    curs = CONN.cursor(pymysql.cursors.DictCursor)
    insert_sql = """insert into player_stat (player_id, match_id, stat_type, stat_value, half) values (%s, %s, %s, %s, %s)"""
    curs.execute(insert_sql, (player_stat['player_id'], player_stat['match_id'], player_stat['stat_type'], player_stat['stat_value'], player_stat['half'],))
    CONN.commit()

In [4]:
root_dirs =['/Users/gracelee/football_worldcup', #group phase
            '/Users/gracelee/football_worldcup_knock']

In [ ]:
for _dir in root_dirs:
    onlyfiles = [f for f in listdir(_dir) if isfile(join(_dir, f))]
    onlyfiles = [_dir+'/'+x for x in onlyfiles if ('teamstatistics' in x and 'fwc' not in x)]

    for z in range(len(onlyfiles)):
        print('file z::: %s' % str(z))
        print(onlyfiles[z])
        player_statistic = textract.process(onlyfiles[z])
        lines = str(player_statistic).split('\\n')
        country_id = get_country('%'+lines[9].lower().strip()+'%')
        if country_id is None:
            country_id = get_country('%'+lines[13].lower().strip()+'%')
        country_id = country_id['id']
        if lines[7].find('#')>=0:
            match_id = lines[7].split(' ')[1]
        else:
            match_id = lines[11].split(' ')[1]
    #     print('%s :::: %s or %s' % (country_id, lines[9], lines[13]))

        df = tabula.read_pdf(onlyfiles[z], multiple_tables=False)    
    #     print(df.iloc[1].values.tolist())
    #     print(df.columns.__len__())

        col_criteria = df.iloc[1].values.tolist()
        merged_list = []
    #     display(df)
        print("-------")

        if col_criteria.__contains__('Total'):
            if col_criteria[2] == 'Total':
                merged_list.append(pd.DataFrame(df.iloc[2:16, [0,1]].values.tolist(), columns = ['#', 'Name']))
                merged_list.append(pd.DataFrame(df[2:16]['Time Distance covered (metres)'].apply(lambda x: list(filter(None, x.split(' ')))).values.tolist(), columns = ['Time', 'Total']))
                merged_list.append(pd.DataFrame(df[2:16]['Unnamed: 3'].apply(lambda x: list(filter(None, x.split(' ')))).values.tolist(), columns = ['In Poss', 'Not in Poss']))
            elif col_criteria[3] == 'Total':
                merged_list.append(pd.DataFrame(df.iloc[2:16, [0,1,2]].values.tolist(), columns = ['#', 'Name', 'Time']))
                merged_list.append(pd.DataFrame(df[2:16]['Unnamed: 4'].apply(lambda x: list(filter(None, x.split(' ')))).values.tolist(), columns = ['In Poss', 'Not in Poss']))

        else:
            merged_list.append(pd.DataFrame(df.iloc[2:16, [0,1,2]].values.tolist(), columns = ['#', 'Name', 'Time']))    
            merged_list.append(pd.DataFrame(df[2:16]['Distance covered (metres)'].apply(lambda x: list(filter(None, x.split(' ')))[:3]).values.tolist(), columns = ['Total', 'In Poss', 'Not in Poss']))


        if col_criteria.__contains__('O.H. A.3rd P.A.'):
            merged_list.append(pd.DataFrame(df[2:16]['Time spent'].apply(lambda x: list(filter(None, str(x).split(' ')))[:3]).values.tolist(), columns = ['O.H.', 'A.3rd', 'P.A.']))
        else:
            si = col_criteria.index('O.H.')
            ei = si + 3
            merged_list.append(pd.DataFrame(pd.DataFrame(df.iloc[2:16, si:ei].values.tolist(), columns = ['O.H.', 'A.3rd', 'P.A.'])))

        time_stat = pd.concat(merged_list, axis=1)
    #     display(time_stat)

        for index, row in time_stat.iterrows():
            num = row['#']
            print(num)
            if num != '':
                try:
                    player_id = get_player_id(country_id, num.strip())['id']
        #             print(get_player_id(country_id, num.strip())['name'])
                    if pd.notna(row['In Poss'].strip()) and row['In Poss']!='None':
        #                 print(row['In Poss'])
                        save_stat({'player_id':player_id,
                                    'match_id':match_id,
                                  'stat_type':'DCIP',
                                  'stat_value':row['In Poss'],
                                  'half':'total'})

                    if pd.notna(row['Not in Poss'].strip()) and row['Not in Poss']!='None':
        #                 print(row['Not in Poss'])
                        save_stat({'player_id':player_id,
                                   'match_id':match_id,
                                  'stat_type':'DCNIP',
                                  'stat_value':row['Not in Poss'],
                                  'half':'total'})
                    if pd.notna(row['O.H.']) and row['O.H.']!='None' and row['O.H.']!='nan':
        #                 print(row['O.H.'])
                        save_stat({'player_id':player_id,
                                   'match_id':match_id,
                                  'stat_type':'TOH',
                                  'stat_value':row['O.H.'],
                                  'half':'total'})
                    if pd.notna(row['A.3rd']) and row['A.3rd']!='None' and row['A.3rd']!='nan':
        #                 print(row['A.3rd'])
                        save_stat({'player_id':player_id,
                                   'match_id':match_id,
                                  'stat_type':'TA3RD',
                                  'stat_value':row['A.3rd'],
                                  'half':'total'})
                    if pd.notna(row['P.A.']) and row['P.A.']!='None' and row['P.A.']!='nan':
        #                 print(row['P.A.'])
                        save_stat({'player_id':player_id,
                                   'match_id':match_id,
                                  'stat_type':'TPA',
                                  'stat_value':row['P.A.'],
                                  'half':'total'})
                except Exception as e:
                    print(e)
                    print('================== player name missing =================')
                    print(row)                
            else:
                print('================== player name missing =================')
                print(row)